# Reinforcement Learning Overview

This notebook implements value iteration and policy iteration.

## Markov Decision Process - MDP

In MDP, there is an agent. The agent choose an action $a_{t}$ at time $t$ and as a consequence, the environment changes.
Here the evniorment is world around the agent. After taking the action, the environment state changes to $s_{t+1}$.
A reward might be emitted associated with what just happened and then this process repeats. ![](nb_images/mdp.png)

So, there is a feedback cycle in that the next action you take, the next decision you make is in a situation that's the consiquence of what you did before.

## 1. Import libraries

In [1]:
import numpy as np
import gymnasium as gym
import gymnasium.spaces as spaces
import time

In [2]:
# action mapping for display the final result
action_mapping = {
    3: '\u2191', # UP
    2: '\u2192', # RIGHT
    1: '\u2193', # DOWN
    0: '\u2190' # LEFT
}
print(' '.join([action_mapping[i] for i in range(4)]))

← ↓ → ↑


## 2. Setup GYM Env for playing

We define a function that will take a GYM environment and plays number of games according to given policy.

In [3]:
def play_episodes(environment, n_episodes, policy, random = False, categorial = False):
    """
    This fucntion plays the given number of episodes given by following a policy or sample randomly from action_space.
    
    Parameters:
        environment: openAI GYM object
        n_episodes: number of episodes to run
        policy: Policy to follow while playing an episode
        random: Flag for taking random actions. if True no policy would be followed and action will be taken randomly
        
    Return:
        wins: Total number of wins playing n_episodes
        total_reward: Total reward of n_episodes
        avg_reward: Average reward of n_episodes
    
    """
    # intialize wins and total reward
    wins = 0
    total_reward = 0
    
    # loop over number of episodes to play
    for episode in range(n_episodes):
        
        # flag to check if the game is finished
        terminated = False
        
        # reset the environment every time when playing a new episode
        state = environment.reset()[0]

        while not terminated:
            
            # check if the random flag is not true then follow the given policy other wise take random action
            if random:
                action = environment.action_space.sample()
            else:
                if categorial:
                    action = np.random.choice(len(policy[state]), p=policy[state])
                else:
                    action = policy[state] 

            # take the next step
            next_state, reward,  terminated, info = environment.step(action)[:-1]
            
            environment.render()
            
            # accumalate total reward
            total_reward += reward
            
            # change the state
            state = next_state
            
            # if game is over with positive reward then add 1.0 in wins
            if terminated and reward == 1.0:
                wins += 1
                
    # calculate average reward
    average_reward = total_reward / n_episodes
    
    return wins, total_reward, average_reward
            

## 3. Solve for Value Iteration.

![](nb_images/value_iter.png)

In [4]:

def one_step_lookahead(env, state, V , discount_factor = 0.99):
    """
    Helper function to  calculate state-value function
    
    Arguments:
        env: openAI GYM environment object
        state: state to consider
        V: Estimated Value for each state. Vector of length nS
        discount_factor: MDP discount factor
        
    Return:
        action_values: Expected value of each action in a state. Vector of length nA
    """
    
    # initialize vector of action values
    action_values = np.zeros(env.action_space.n)
    
    # loop over the actions we can take in an environment 
    for action in range(env.action_space.n):
        # loop over the P_sa distribution.
        for probablity, next_state, reward, info in env.P[state][action]:
             #if we are in state s and take action a. then sum over all the possible states we can land into.
            action_values[action] += probablity * (reward + (discount_factor * V[next_state]))
            
    return action_values

In [5]:
def update_policy(env, policy, V, discount_factor):
    
    """
    Helper function to update a given policy based on given value function.
    
    Arguments:
        env: openAI GYM environment object.
        policy: policy to update.
        V: Estimated Value for each state. Vector of length nS.
        discount_factor: MDP discount factor.
    Return:
        policy: Updated policy based on the given state-Value function 'V'.
    """
    
    for state in range(env.observation_space.n):
        # for a given state compute state-action value.
        action_values = one_step_lookahead(env, state, V, discount_factor)
        
        # choose the action which maximizez the state-action value.
        policy[state] =  np.argmax(action_values)
        
    return policy
    

In [6]:
def value_iteration(env, discount_factor = 0.999, max_iteration = 1000):
    """
    Algorithm to solve MPD.
    
    Arguments:
        env: openAI GYM environment object.
        discount_factor: MDP discount factor.
        max_iteration: Maximum No.  of iterations to run.
        
    Return:
        V: Optimal state-Value function. Vector of lenth nS.
        optimal_policy: Optimal policy. Vector of length nS.
    
    """
    # intialize value fucntion
    V = np.zeros(env.observation_space.n)
    
    # iterate over max_iterations
    for i in range(max_iteration):
        
        #  keep track of change with previous value function
        prev_v = np.copy(V) 
    
        # loop over all states
        for state in range(env.observation_space.n):
            
            # Asynchronously update the state-action value
            #action_values = one_step_lookahead(env, state, V, discount_factor)
            
            # Synchronously update the state-action value
            action_values = one_step_lookahead(env, state, prev_v, discount_factor)
            
            # select best action to perform based on highest state-action value
            best_action_value = np.max(action_values)
            
            # update the current state-value fucntion
            V[state] =  best_action_value
            
        # if policy not changed over 10 iterations it converged.
        if i % 10 == 0:
            # if values of 'V' not changing after one iteration
            if (np.all(np.isclose(V, prev_v))):
                print('Value converged at iteration %d' %(i+1))
                break

    # intialize optimal policy
    optimal_policy = np.zeros(env.observation_space.n, dtype = 'int8')
    
    # update the optimal polciy according to optimal value function 'V'
    optimal_policy = update_policy(env, optimal_policy, V, discount_factor)
    
    return V, optimal_policy

## Test the Algorithim

In [7]:
environment = gym.make('FrozenLake-v1')
tic = time.time()
opt_V, opt_Policy = value_iteration(environment.env, max_iteration = 1000)
toc = time.time()
elapsed_time = (toc - tic) * 1000
print (f"Time to converge: {elapsed_time: 0.3} ms")
print('Optimal Value function: ')
print(opt_V.reshape((4, 4)))
print('Final Policy: ')
print(opt_Policy)
print(' '.join([action_mapping[int(action)] for action in opt_Policy]))

Value converged at iteration 341
Time to converge:  1.27e+02 ms
Optimal Value function: 
[[0.78538826 0.77836049 0.77368481 0.7713498 ]
 [0.78775777 0.         0.50562724 0.        ]
 [0.79250312 0.79963699 0.74472318 0.        ]
 [0.         0.86409247 0.93114742 0.        ]]
Final Policy: 
[0 3 3 3 0 0 0 0 3 1 0 0 0 2 1 0]
← ↑ ↑ ↑ ← ← ← ← ↑ ↓ ← ← ← → ↓ ←


/home/jackkert/anaconda3/envs/DRL_project/lib/python3.12/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.P to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.P` for environment variables or `env.get_wrapper_attr('P')` that will search the reminding wrappers.
  logger.warn(


In [8]:
n_episode = 10
wins, total_reward, avg_reward = play_episodes(environment, n_episode, opt_Policy, random = False)

/home/jackkert/anaconda3/envs/DRL_project/lib/python3.12/site-packages/gymnasium/envs/toy_text/frozen_lake.py:328: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym.make("FrozenLake-v1", render_mode="rgb_array")
  gym.logger.warn(


In [9]:
print(f'Total wins with value iteration: {wins}')
print(f"Average rewards with value iteration: {avg_reward}")

Total wins with value iteration: 9
Average rewards with value iteration: 0.9


## 4. Solve for Policy Iteration

![](nb_images/policy_iter.png)

In [10]:
def policy_eval(env, policy, V, discount_factor):
    """
    Helper function to evaluate a policy.
    
    Arguments:
        env: openAI GYM environment object.
        policy: policy to evaluate.
        V: Estimated Value for each state. Vector of length nS.
        discount_factor: MDP discount factor.
    Return:
        policy_value: Estimated value of each state following a given policy and state-value 'V'. 
        
    """
    policy_value = np.zeros(env.observation_space.n)
    for state in range(env.observation_space.n):
        for action, action_prob in enumerate(policy[state]): # evaluate policy value over all actions
            for probablity, next_state, reward, info in env.P[state][action]:
                policy_value[state] += action_prob * probablity * (reward + (discount_factor * V[next_state]))
    
    return policy_value

In [11]:
# testing policy_eval for categorial policy
env_test = gym.make('FrozenLake-v1')
a = np.random.randint(0, 1000, (env_test.observation_space.n, env_test.action_space.n))
policy_test = a/a.sum(axis = 1, keepdims = True)
V_test = np.zeros(env_test.observation_space.n)
discount_factor_test = 0.9999

policy_eval(env_test, policy_test, V_test, discount_factor_test)


array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.22358262,
       0.        ])

In [12]:
def one_step_lookahead_categorial(env, state, V , discount_factor = 0.99):
    """
    Helper function to  calculate state-value function
    
    Arguments:
        env: openAI GYM environment object
        state: state to consider
        V: Estimated Value for each state. Vector of length nS
        discount_factor: MDP discount factor
        
    Return:
        action_values: Expected value of each action in a state. Vector of length nA
    """
    
    # initialize vector of action values
    action_values = np.zeros(env.action_distributions.shape[0]) 
    # loop over the actions we can take in an environment 
    for action_dist_id, action_distribution in enumerate(env.action_distributions): 
        # loop over the P_sa distribution.
        for action, action_prob in enumerate(action_distribution):
            for probablity, next_state, reward, info in env.P[state][action]:
                 #if we are in state s and take action a. then sum over all the possible states we can land into.
                action_values[action_dist_id] += action_prob * probablity * (reward + (discount_factor * V[next_state]))
            
    return action_values

In [13]:
def update_policy_categorial(env, policy, V, discount_factor):
    
    """
    Helper function to update a given policy based on given value function.
    
    Arguments:
        env: openAI GYM environment object.
        policy: categorial policy to update.
        V: Estimated Value for each state. Vector of length nS.
        discount_factor: MDP discount factor.
    Return:
        policy: Updated policy based on the given state-Value function 'V'.
    """
    
    for state in range(env.observation_space.n):
        # for a given state compute state-action value.
        action_values = one_step_lookahead_categorial(env, state, V, discount_factor)
        # choose the action which maximizez the state-action value.
        id_max_action_value = np.argmax(action_values)
        policy[state] = env.action_distributions[id_max_action_value]
        
    return policy
    

In [14]:
def policy_iteration(env, discount_factor = 0.999, max_iteration = 1000):
    """
    Algorithm to solve MPD.
    
    Arguments:
        env: openAI GYM environment object.
        discount_factor: MDP discount factor.
        max_iteration: Maximum No.  of iterations to run.
        
    Return:
        V: Optimal state-Value function. Vector of lenth nS.
        new_policy: Optimal policy. Vector of length nS.
    
    """
    # intialize the state-Value function
    V = np.zeros(env.observation_space.n)

    # create N random action distributions in tabular fashion:
    N_dists = 10000
    random_actions = np.random.randint(0, 1000, (N_dists, env.action_space.n))
    env.action_distributions = random_actions/random_actions.sum(axis = 1, keepdims = True)
    # intialize a random implicit categorial policy
    dists_idx = np.arange(N_dists)
    np.random.shuffle(dists_idx)
    policy = env.action_distributions[dists_idx[:env.observation_space.n]]
    policy_prev = np.copy(policy)
    for i in range(max_iteration):
        # evaluate given policy
        V = policy_eval(env, policy, V, discount_factor)
        
        # improve policy
        policy = update_policy_categorial(env, policy, V, discount_factor)
        
        # if policy not changed over 10 iterations it converged.
        if i % 10 == 0:
            if (np.all(np.equal(policy, policy_prev))):
                print('policy converged at iteration %d' %(i+1))
                break
            policy_prev = np.copy(policy)
            

            
    return V, policy

## Test Policy Iteration

In [15]:
environment2 = gym.make('FrozenLake-v1') #, render_mode = "human")
tic = time.time()
opt_V2, opt_policy2 = policy_iteration(environment2.env, discount_factor = 0.999, max_iteration = 10000)
toc = time.time()
elapsed_time = (toc - tic) * 1000
print (f"Time to converge: {elapsed_time: 0.3} ms")
print('Optimal Value function: ')
print(opt_V2.reshape((4, 4)))
print('Final Policy: ')
print(opt_policy2)
# Display the most likely policy in discrete action
print("Most likely actions taken by policy: ")
print(' '.join([action_mapping[(np.argmax(action))] for action in opt_policy2]))

policy converged at iteration 41
Time to converge:  1.28e+05 ms
Optimal Value function: 
[[0.25857573 0.22056746 0.21555792 0.18758814]
 [0.2731765  0.         0.22067219 0.        ]
 [0.32858422 0.43530956 0.45475947 0.        ]
 [0.         0.5726475  0.76849063 0.        ]]
Final Policy: 
[[0.88762984 0.01605288 0.06515581 0.03116147]
 [0.08144192 0.03204272 0.         0.88651535]
 [0.88762984 0.01605288 0.06515581 0.03116147]
 [0.08144192 0.03204272 0.         0.88651535]
 [0.88762984 0.01605288 0.06515581 0.03116147]
 [0.10220441 0.29579158 0.36873747 0.23326653]
 [0.40139616 0.01628854 0.57998837 0.00232693]
 [0.10220441 0.29579158 0.36873747 0.23326653]
 [0.08144192 0.03204272 0.         0.88651535]
 [0.0523416  0.88888889 0.0523416  0.00642792]
 [0.88762984 0.01605288 0.06515581 0.03116147]
 [0.10220441 0.29579158 0.36873747 0.23326653]
 [0.10220441 0.29579158 0.36873747 0.23326653]
 [0.0254842  0.03975535 0.90519878 0.02956167]
 [0.01088032 0.87240356 0.0504451  0.06627102]
 [

In [16]:
n_episode = 10
wins, total_reward, avg_reward = play_episodes(environment2, n_episode, opt_policy2, random = False, categorial = True)

In [17]:
print(f'Total wins with Policy iteration: {wins}')
print(f"Average rewards with Policy iteration: {avg_reward}")

Total wins with Policy iteration: 3
Average rewards with Policy iteration: 0.3


## Remarks

Policy Iteration converge faster but takes more computation.